In [154]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.metrics as metrics
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score


In [133]:
# 1. read the ONS file
df = pd.read_excel("data/internetreferencetables.xlsx",
                   sheet_name="IntValSA", header=3)


In [134]:
df.head()

,Time Period,All retailing excluding automotive fuel [Note 1],Predominantly food stores,Total of predominantly non-food stores [Note 2],Non-specialised stores [Note 2],"Textile, clothing and footwear stores [Note 2]",Household goods stores [Note 2],Other stores [Note 2],Non-store retailing
0,Agg/SIC,Agg 21X,Agg 1,Agg 12,47.19,Agg 5,Agg 7,Agg 13,Agg 14
1,Dataset identifier code,MZX6,MZX7,MZX8,MZX9,MZXV,MZY2,MZY3,MZY4
2,2008 Jan,219.5,36.9,91.6,16.4,23.9,21.6,29.7,91
3,2008 Feb,231.6,37.8,97.9,17.8,26,23.2,31,95.9
4,2008 Mar,238.3,37.7,99.5,18,26.6,24.3,30.5,101.2


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 9 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Time Period                                       215 non-null    object
 1   All retailing excluding automotive fuel [Note 1]  215 non-null    object
 2   Predominantly food stores                         215 non-null    object
 3   Total of predominantly non-food stores [Note 2]   215 non-null    object
 4   Non-specialised stores [Note 2]                   215 non-null    object
 5   Textile, clothing and footwear stores [Note 2]    215 non-null    object
 6   Household goods stores [Note 2]                   215 non-null    object
 7   Other stores [Note 2]                             215 non-null    object
 8   Non-store retailing                               215 non-null    object
dtypes: object(9)
memory usage: 15.2+ 

In [136]:
df.shape

(215, 9)

### Data Cleaning

In [137]:
# Rename columns
df.columns = ["time_period", "all_retail_excl_autofuel", "food_stores", "non_food_stores_total", "non_specialised_stores", "textile_clothing_footwear_stores", "household_goods_stores", "other_stores", "non-store_retail"]

In [138]:
# Drop first 2 rows
df = df.iloc[2:].reset_index(drop=True)

In [139]:
df.head()

,time_period,all_retail_excl_autofuel,food_stores,non_food_stores_total,non_specialised_stores,textile_clothing_footwear_stores,household_goods_stores,other_stores,non-store_retail
0,2008 Jan,219.5,36.9,91.6,16.4,23.9,21.6,29.7,91
1,2008 Feb,231.6,37.8,97.9,17.8,26,23.2,31,95.9
2,2008 Mar,238.3,37.7,99.5,18,26.6,24.3,30.5,101.2
3,2008 Apr,247,38.8,104,18.6,27.2,25.4,32.9,104.2
4,2008 May,266.1,40.3,106.3,19.3,28.4,26.1,32.5,119.5


In [140]:
# Remove spaces after date
df['time_period'] = df['time_period'].str.strip()

# Data conversions
df['time_period'] = pd.to_datetime(df['time_period'], format="%Y %b")

cols = df.columns[1:]
df[cols] = df[cols].apply(pd.to_numeric, errors="coerce")

In [141]:
# Melt the df
id_cols = ["time_period"]  # keep the date
value_cols = cols

df_long = df.melt(
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="store_type",
    value_name="internet_value"
)

### Reshaping and new columns

In [152]:
df_long.head(10)

,time_period,store_type,internet_value,time,last_month_value,last_year_value,diff_1_month,covid,is_food_stores,is_household_goods_stores,is_non-store_retail,is_non_food_stores_total,is_non_specialised_stores,is_other_stores,is_textile_clothing_footwear_stores
12,2009-01-01,all_retail_excl_autofuel,293.0,13,285.9,219.5,7.1,0,False,False,False,False,False,False,False
13,2009-02-01,all_retail_excl_autofuel,312.4,14,293.0,231.6,19.4,0,False,False,False,False,False,False,False
14,2009-03-01,all_retail_excl_autofuel,311.7,15,312.4,238.3,-0.7,0,False,False,False,False,False,False,False
15,2009-04-01,all_retail_excl_autofuel,325.0,16,311.7,247.0,13.3,0,False,False,False,False,False,False,False
16,2009-05-01,all_retail_excl_autofuel,333.5,17,325.0,266.1,8.5,0,False,False,False,False,False,False,False
17,2009-06-01,all_retail_excl_autofuel,335.9,18,333.5,270.5,2.4,0,False,False,False,False,False,False,False
18,2009-07-01,all_retail_excl_autofuel,359.7,19,335.9,281.4,23.8,0,False,False,False,False,False,False,False
19,2009-08-01,all_retail_excl_autofuel,358.9,20,359.7,289.0,-0.8,0,False,False,False,False,False,False,False
20,2009-09-01,all_retail_excl_autofuel,356.3,21,358.9,294.5,-2.6,0,False,False,False,False,False,False,False
21,2009-10-01,all_retail_excl_autofuel,376.0,22,356.3,280.9,19.7,0,False,False,False,False,False,False,False


In [147]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1704 entries, 0 to 1064
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   time_period                          1704 non-null   datetime64[ns]
 1   store_type                           1704 non-null   object        
 2   internet_value                       1704 non-null   float64       
 3   time                                 1704 non-null   int64         
 4   last_month_value                     1696 non-null   float64       
 5   last_year_value                      1608 non-null   float64       
 6   diff_1_month                         1696 non-null   float64       
 7   covid                                1704 non-null   int64         
 8   is_food_stores                       1704 non-null   bool          
 9   is_household_goods_stores            1704 non-null   bool          
 10  is_non-store_reta

In [144]:
# Add month count
df_long = df_long.sort_values(['store_type', 'time_period'])
df_long['time'] = df_long.groupby('store_type').cumcount() + 1 # Number months for each sector, how far along in time we are

# Add the actual value from the previous month for the same store type. If Jan was high, Feb is likely high too. That’s short-term persistence.
df_long['last_month_value'] = df_long.groupby('store_type')['internet_value'].shift(1)

# Add last year value (the same month). For “Food stores – Feb 2015” we look up “Food stores – Feb 2014”.
df_long['last_year_value'] = df_long.groupby('store_type')['internet_value'].shift(12)

# Add the difference
df_long['diff_1_month'] = df_long['internet_value'] - df_long['last_month_value']


In [145]:
# Create a dummy column (0,1) for Covid shock
df_long['covid'] = (
    (df_long['time_period'] >= "2020-04-01") &
    (df_long['time_period'] <= "2021-03-01")
).astype(int)

In [146]:
# Create store type dummies except 'all_retail_excl_autofuel', it's baseline
store_dum = pd.get_dummies(df_long['store_type'], prefix="is", drop_first=True)
df_long = pd.concat([df_long, store_dum], axis=1)

In [151]:
# Drop rows that can't have last_year_value and last_month_value (Linear regression can’t use rows with NaN in the features)
df_long = df_long.dropna(subset=['last_month_value', 'last_year_value'])

### Linear Regression

In [153]:
# Create X and y
feature_cols = ["time", "last_month_value", "last_year_value", "covid"] + \
               [c for c in df_long.columns if c.startswith("is_")]
X = df_long[feature_cols]
y = df_long["internet_value"]

In [ ]:
# Fit linear regression
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False  # keep time order
)

model = LinearRegression().fit(X_train, y_train)
y_pred_test = model.predict(X_test)